In [28]:
%load_ext autoreload
%autoreload 2
from knowme.ingest import NotionIngestor
from knowme.embedder import ChromaEmbedder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [57]:
ingestor = NotionIngestor(
    folder_path="/Users/abhinavkashyap/abhi/projects/knowme/abhinav_notion"
)
texts, metadatas = ingestor.ingest()

In [58]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

In [59]:
documents = text_splitter.create_documents(texts=texts, metadatas=metadatas)

In [60]:
print(len(documents))
documents[10].page_content

38


"Let's explore this paper to better understand the claimed advantages and improvements over GPT-3.\n\n## Mixture of Experts\n\n![Screenshot 2023-11-27 at 5.38.08 PM.png](GLaM%20Efficient%20Scaling%20of%20Language%20Models%20with%20Mix%20daa1c17d134249129c43a311d3ff31c5/Screenshot_2023-11-27_at_5.38.08_PM.png)\n\nThis is a transformer block. The major computational bottleneck in Transformers is the FFN or the Feedforward Neural Network Component. Instead of a single FFN layer, can we replace it with multiple FFN modules? We can. These individual FFN modules are called experts. Doing this can have multiple advantages:"

In [61]:
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings()
store = ChromaEmbedder(text_splitter, embedding_function=embedding_function)

In [62]:
chromastore = store.store_embeddings(texts, metadatas)

In [63]:
retriever = chromastore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What does Abhinav work as?")

In [65]:
# Some of the information is there in the first retrieved document
# I might have to do some cleaning on the ingestion side
retrieved_docs[1]

Document(page_content='[https://notaku.so/iframes/clickable-image/eyJpbWFnZVVybCI6Imh0dHBzOi8vZW5jcnlwdGVkLXRibjAuZ3N0YXRpYy5jb20vaW1hZ2VzP3E9dGJuOkFOZDlHY1RMV0FlbWMzMFZRZkluMEFuU3AybDE1dUR2WXptYUZfdU1YZyZ1c3FwPUNBVSIsImxpbmtIcmVmIjoiaHR0cHM6Ly90d2l0dGVyLmNvbS9hYmthc2h5YXA5MiJ9](https://notaku.so/iframes/clickable-image/eyJpbWFnZVVybCI6Imh0dHBzOi8vZW5jcnlwdGVkLXRibjAuZ3N0YXRpYy5jb20vaW1hZ2VzP3E9dGJuOkFOZDlHY1RMV0FlbWMzMFZRZkluMEFuU3AybDE1dUR2WXptYUZfdU1YZyZ1c3FwPUNBVSIsImxpbmtIcmVmIjoiaHR0cHM6Ly90d2l0dGVyLmNvbS9hYmthc2h5YXA5MiJ9)\n\nHey I’m Abhinav. I am a AI Scientist at ASUS-AICS. I work on Natural Language Processing and I am  interested in processing clinical notes.', metadata={'filename': '/Users/abhinavkashyap/abhi/projects/knowme/abhinav_notion/Abhinav Ramesh Kashyap 2a5b5c4c70f240359dc297eaa025b430.md', 'start_index': 0})